In [ ]:
# This is a conceptual Jupyter Notebook for Exploratory Data Analysis (EDA)
# of your processed egress cost data.
# You would run this on your SageMaker Notebook Instance.

# --- Section 1: Setup and Data Loading ---
import sagemaker
import boto3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Initialize SageMaker session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket() # Or your specific processed data bucket

# Define S3 path to processed data (output from Glue jobs)
# This should match the target_path in your Glue job configuration.
processed_data_s3_path = f"s3://{bucket}/processed_egress_costs/aggregated_egress_costs/" # Example path for CUR data
flow_log_data_s3_path = f"s3://{bucket}/processed_egress_costs/aggregated_flow_data/" # Example path for Flow Log data

print(f"Loading processed egress costs from: {processed_data_s3_path}")
print(f"Loading processed flow log data from: {flow_log_data_s3_path}")

# Load processed data (e.g., daily aggregated egress costs from CUR)
# Ensure your Glue job outputs daily aggregated egress data in Parquet format.
try:
    df_egress_costs = sagemaker_session.read_parquet(processed_data_s3_path)
    print(f"Loaded egress costs data with shape: {df_egress_costs.shape}")
    print(df_egress_costs.head())
except Exception as e:
    print(f"Warning: Could not load egress costs data. Make sure Glue job has run and path is correct: {e}")
    df_egress_costs = pd.DataFrame()

# Load processed flow log data (e.g., hourly aggregated flow data)
try:
    df_flow_logs = sagemaker_session.read_parquet(flow_log_data_s3_path)
    print(f"Loaded flow logs data with shape: {df_flow_logs.shape}")
    print(df_flow_logs.head())
except Exception as e:
    print(f"Warning: Could not load flow logs data. Make sure Glue job has run and path is correct: {e}")
    df_flow_logs = pd.DataFrame()


# --- Section 2: Exploratory Data Analysis (EDA) ---

if not df_egress_costs.empty:
    print("\n--- EDA for Aggregated Egress Costs ---")
    df_egress_costs['usage_date'] = pd.to_datetime(df_egress_costs['usage_date'])
    df_egress_costs = df_egress_costs.sort_values('usage_date')

    # Visualize daily egress cost trends
    plt.figure(figsize=(18, 7))
    plt.plot(df_egress_costs['usage_date'], df_egress_costs['daily_egress_cost_usd'], label='Daily Egress Cost (USD)')
    plt.title('Daily Egress Cost Over Time')
    plt.xlabel('Date')
    plt.ylabel('Cost (USD)')
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

    # Visualize egress cost by service
    plt.figure(figsize=(12, 6))
    sns.barplot(x='service_code', y='daily_egress_cost_usd', data=df_egress_costs.groupby('service_code').sum().reset_index())
    plt.title('Total Egress Cost by Service')
    plt.xlabel('Service Code')
    plt.ylabel('Total Cost (USD)')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

    # Box plot to identify outliers in cost
    plt.figure(figsize=(8, 6))
    sns.boxplot(y=df_egress_costs['daily_egress_cost_usd'])
    plt.title('Box Plot of Daily Egress Cost')
    plt.ylabel('Cost (USD)')
    plt.tight_layout()
    plt.show()

if not df_flow_logs.empty:
    print("\n--- EDA for Aggregated Flow Logs ---")
    df_flow_logs['flow_date'] = pd.to_datetime(df_flow_logs['flow_date'])
    df_flow_logs = df_flow_logs.sort_values(['flow_date', 'flow_hour'])

    # Visualize hourly egress bytes from flow logs
    plt.figure(figsize=(18, 7))
    df_flow_logs['datetime'] = df_flow_logs['flow_date'] + pd.to_timedelta(df_flow_logs['flow_hour'], unit='h')
    plt.plot(df_flow_logs['datetime'], df_flow_logs['total_egress_bytes'], label='Hourly Egress Bytes')
    plt.title('Hourly Egress Bytes from VPC Flow Logs Over Time')
    plt.xlabel('Datetime')
    plt.ylabel('Bytes')
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

    # Top 10 destination IPs by egress bytes
    plt.figure(figsize=(12, 7))
    top_dest_ips = df_flow_logs.groupby('destination_ip')['total_egress_bytes'].sum().nlargest(10).reset_index()
    sns.barplot(x='destination_ip', y='total_egress_bytes', data=top_dest_ips)
    plt.title('Top 10 Destination IPs by Egress Bytes')
    plt.xlabel('Destination IP')
    plt.ylabel('Total Egress Bytes')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

# --- Section 3: Feature Engineering Preview (Conceptual) ---
# This section would demonstrate how to apply the feature_engineering.py logic
# to your combined data, showing the resulting features.
# You would typically combine df_egress_costs and df_flow_logs here based on date/time.

# Example of combining (conceptual)
df_combined = pd.merge(df_egress_costs, df_flow_logs, left_on='usage_date', right_on='flow_date', how='left')
print("\nCombined DataFrame head (conceptual):")
print(df_combined.head())

# This notebook helps to understand your data before training a model.